# Sample code for Compare Weights of VGG16 PyTorch and Keras

In [1]:
from collections import OrderedDict
import numpy as np

#import torch
#import torch.nn as nn
import torchvision.models as models

import tensorflow as tf

2024-04-14 13:22:21.972102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 13:22:21.972201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 13:22:21.977155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 13:22:22.000258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load VGG16 model

In [2]:
vgg_pytorch = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

In [3]:
vgg_tf = tf.keras.applications.vgg16.VGG16(weights='imagenet')

2024-04-14 13:22:39.109640: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-14 13:22:39.136377: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-14 13:22:39.136409: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-14 13:22:39.149669: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-14 13:22:39.149706: I external/local_xla/xla/stream_executor

## Load weights

In [4]:
weights_pytorch = OrderedDict()
for name, param in vgg_pytorch.named_parameters():
    weights_pytorch[name] = param
weights_pytorch.keys()

odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])

In [5]:
key = list(weights_pytorch.keys())[0]
weight_val_pytorch = weights_pytorch[key].detach().numpy()

In [6]:
f'[out_ch, in_ch, kernel_h, kernel_w] = {weight_val_pytorch.shape}'

'[out_ch, in_ch, kernel_h, kernel_w] = (64, 3, 3, 3)'

In [7]:
weights_tf = vgg_tf.weights
weight_val_tf = np.array(weights_tf[0].read_value())
f'[kernel_h, kernel_w, in_ch, out_ch] = {weight_val_tf.shape}'

'[kernel_h, kernel_w, in_ch, out_ch] = (3, 3, 3, 64)'

## Compare weights

In [8]:
weight_val_pytorch_tf_fmt = weight_val_pytorch.transpose([2, 3, 1, 0])  # [out_ch, in_ch, kernel_h, kernel_w] -> [kernel_h, kernel_w, in_ch, out_ch]

In [9]:
diff = weight_val_pytorch_tf_fmt - weight_val_tf

In [10]:
diff.min()

-1.8021048

In [11]:
diff.max()

1.1417828

In [12]:
weight_val_pytorch_tf_fmt

array([[[[-5.53730607e-01,  2.32537210e-01,  1.77150413e-01, ...,
           7.75985420e-02,  1.65203176e-02,  3.21764126e-02],
         [ 1.75483912e-01, -1.40762165e-01,  3.09239298e-01, ...,
          -1.83272704e-01,  4.27811295e-02,  2.50632346e-01],
         [ 3.13025624e-01, -3.14323962e-01,  3.15720350e-01, ...,
          -2.25471616e-01,  1.26607120e-01,  4.60319877e-01]],

        [[ 1.42704695e-01,  1.26659840e-01,  5.21490514e-01, ...,
           1.26918361e-01, -3.22253704e-02, -1.07658058e-01],
         [ 9.86296963e-03, -2.19025537e-01,  6.70709550e-01, ...,
          -6.74249977e-02, -6.75286353e-02,  1.55430257e-01],
         [-1.65913403e-01, -3.70477855e-01,  4.23345655e-01, ...,
          -1.19312614e-01,  8.50455090e-02,  4.33990777e-01]],

        [[ 5.28961480e-01,  1.86054602e-01,  9.87398718e-03, ...,
           3.23051959e-02, -3.84501694e-03, -2.63881803e-01],
         [-8.14131498e-02,  1.50406718e-01,  2.05457117e-02, ...,
          -7.21196318e-03, -7.0308

In [13]:
weight_val_tf

array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
          -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
         [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
          -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
         [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
          -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],

        [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
          -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
         [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
          -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
         [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
          -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],

        [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
          -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
         [-8.13870355e-02,  4.18543853e-02, -1.01763301e-01, ...,
          -9.43124294e-02, -5.0566